In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, LSTM, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = yf.download('TSLA')

In [ ]:
def add_technicals(df):
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()
    df['MA200'] = df['Close'].rolling(window=200).mean()
    df['RSI'] = calculate_rsi(df['Close'], 14)
    exp1 = df['Close'].ewm(span=12, adjust=False).mean()
    exp2 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['Signal Line'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['20STD'] = df['Close'].rolling(window=20).std()
    df['Upper Band'] = df['MA20'] + (df['20STD'] * 2)
    df['Lower Band'] = df['MA20'] - (df['20STD'] * 2)
    df['L14'] = df['Low'].rolling(window=14).min()
    df['H14'] = df['High'].rolling(window=14).max()
    df['%K'] = (df['Close'] - df['L14']) * 100 / (df['H14'] - df['L14'])
    df['%D'] = df['%K'].rolling(window=3).mean()
    high = df['High'].max()
    low = df['Low'].min()
    fib_levels = fibonacci_retracement_levels(high, low)
    df['Fib236'] = fib_levels[0]
    df['Fib382'] = fib_levels[1]
    df['Fib618'] = fib_levels[2]
    df['VMA20'] = df['Volume'].rolling(window=20).mean()
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    return df

def calculate_rsi(data, window):
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def fibonacci_retracement_levels(high, low):
    diff = high - low
    level1 = high - 0.236 * diff
    level2 = high - 0.382 * diff
    level3 = high - 0.618 * diff
    return level1, level2, level3

In [ ]:
df = add_technicals(df)
df.dropna(inplace=True)

In [ ]:
# minimalist approach
# testing using minimal features and minimal neural layers
# gradually increasing the feature count as we get better results
features = ['Close', 'MA20','MA50']
df = df[features]

In [ ]:
# gotta remember the scaler for Close as we need to inverse transform the output later - maybe not :)
for col in features:
    scaler = MinMaxScaler()
    if col == 'Close':
        cls_scaler = scaler
    df[col] = scaler.fit_transform(df[[col]])

In [ ]:
# change this horizon as needed
forecast_horizon = 10

df['Close_shft'] = df['Close'].shift(-forecast_horizon)
df.dropna(inplace=True)

In [ ]:
q_80 = int(len(df) * .8)
q_90 = int(len(df) * .9)
train = df[:q_80]
val = df[q_80:q_90]
test = df[q_90:]

In [ ]:
num_features = len(features)
time_stamps = forecast_horizon

In [ ]:
num_features

In [ ]:
X_train, y_train = train.loc[:, train.columns != 'Close_shft'], train['Close_shft']
X_val, y_val = val.loc[:, val.columns != 'Close_shft'], val['Close_shft']
X_test, y_test = test.loc[:, test.columns != 'Close_shft'], test['Close_shft']

In [ ]:
# the best prediction comes from this architecure:
#       LSTM(50) -> Dropout(.1) -> Dense(1, activation='tanh') (30 epochs)
#       with the best rmse of 0.0690
#       and using this X -> ['Close', 'MA20','MA50'] and this y -> ['Close_shft']
#       with the forecast_horizon of 10
model = Sequential([
    LSTM(50, input_shape=(num_features, 1)),
    Dropout(.1),
    Dense(1, activation='tanh'),
])
model.compile(optimizer=Adam(learning_rate=.005), loss='mse')
history = model.fit(x=X_train, y=y_train, batch_size=time_stamps, epochs=30,
                    validation_data=(X_val, y_val))

In [ ]:
y_pred = model.predict(X_test)
y_pred = y_pred.flatten()
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
rmse

In [ ]:
plt.plot(range(len(y_pred)), y_pred, color='red', label='Prediction')
plt.plot(range(len(y_test)), y_test, color='blue', label='Actual')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# testing the model on other stocks
aapl = yf.download('AAPL')
aapl['MA20'] = aapl['Close'].rolling(window=20).mean()
aapl['MA50'] = aapl['Close'].rolling(window=20).mean()
for col in features:
    scaler = MinMaxScaler()
    if col == 'Close':
        cls_scaler = scaler
    aapl[col] = scaler.fit_transform(aapl[[col]])
aapl['Close_shft'] = aapl['Close'].shift(-forecast_horizon)
aapl.dropna(inplace=True)
aapl = aapl[['Close', 'Close_shft', 'MA20', 'MA50']]
y_pred = model.predict(aapl[['Close', 'MA20', 'MA50']])
rmse = np.sqrt(mean_squared_error(aapl['Close_shft'], y_pred))
print(rmse)
plt.plot(y_pred, color='red', label='Prediction')
plt.plot(aapl['Close_shft'], color='blue', label='Actual')
plt.grid()
plt.legend()
plt.show()